In [ ]:
import pandas as pd
from datetime import datetime

## Load datasets

### Covid-19 cases


In [ ]:
df_covid = pd.read_csv('Datasets/covid-19-data-resource-hub-covid-19-case-counts/COVID-19 Activity.csv')

Este dataset pode ser encontrado em: https://data.world/covid-19-data-resource-hub/covid-19-case-counts

O download do arquivo utilizado neste estudo foi realizado em: xxxxx

### Demographic

In [ ]:
df_demographic = pd.read_csv('Datasets/demographic/demographic-2019.csv',  encoding='latin-1')

Este dataset pode ser encontrado em: https://data.world/hdx/749ed4a9-6a89-4a3f-a4c8-b5359966a6e9
        
O download do arquivo utilizado neste estudo foi realizado em: xxxxx

In [ ]:
df_demographic = df_demographic.loc[:, df_demographic.columns.intersection([
                                         'ISO', 
                                         'Total population in millions, 2019',
                                         'Total fertility rate, per woman, 2019',
                                         'Population aged 0-14, percent, 2019',
                                         'Population aged 10-24, percent, 2019',
                                         'Population aged 15-64, percent, 2019',
                                         'Population aged 65 and older, percent, 2019',
                                         'Life expectancy at birth, years, 2019'])]

In [ ]:
df_demographic = df_demographic.rename(columns={"Total population in millions, 2019": "TOTAL_POPULATION", 
                                                "Total fertility rate, per woman, 2019": "FERTILITY_RATE",
                                                "Population aged 0-14, percent, 2019": "PERCENT_POPULATION_AGED_0_14",
                                                "Population aged 10-24, percent, 2019": "PERCENT_POPULATION_AGED_10_24",
                                                "Population aged 15-64, percent, 2019": "PERCENT_POPULATION_AGED_15_64",
                                                "Population aged 65 and older, percent, 2019": "PERCENT_POPULATION_AGED_65_OLDER"
                                               })

### Cases metricts

In [ ]:
df_covid['REPORT_DATE'] = pd.to_datetime(df_covid['REPORT_DATE'], format='%m/%d/%Y')

In [ ]:
df_covid = df_covid.set_index('REPORT_DATE')

In [ ]:
countries = df_covid['COUNTRY_SHORT_NAME'].unique()

In [ ]:
 for country in countries:
        day_0_for_death = df_covid[(df_covid['COUNTRY_SHORT_NAME'] == country) & (df_covid['PEOPLE_DEATH_NEW_COUNT']>0)].index.min()
        day_0_for_cases = df_covid[(df_covid['COUNTRY_SHORT_NAME'] == country) & (df_covid['PEOPLE_POSITIVE_NEW_CASES_COUNT']>0)].index.min()
        df_covid.loc[df_covid['COUNTRY_SHORT_NAME'] == country, 'DAYS_AFTER_FIRST_DEATH'] = df_covid[df_covid['COUNTRY_SHORT_NAME'] == country].apply(lambda x: (x.name - day_0_for_death).days if (x.name - day_0_for_death).days >0 else None, axis=1)
        df_covid.loc[df_covid['COUNTRY_SHORT_NAME'] == country, 'DAYS_AFTER_FIRST_CASE'] = df_covid[df_covid['COUNTRY_SHORT_NAME'] == country].apply(lambda x: (x.name - day_0_for_cases).days if (x.name - day_0_for_cases).days >0 else None, axis=1)
        

### FINAL DATASET

In [ ]:
df = pd.merge(df_covid, df_demographic, how='left', left_on='COUNTRY_ALPHA_2_CODE', right_on='ISO')

## Brazil dataset

In [ ]:
ds_brazil = df_covid[df_covid['COUNTRY_SHORT_NAME'] == 'Brazil']